In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 
from scipy.interpolate import RegularGridInterpolator

%matplotlib inline
import matplotlib.pyplot as plt

from pyfrac.utils.sofiutils import *

# FILE PATHS

In [2]:
sofimaster = '/home/birniece/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/sofi3D'

In [3]:
# Project Directory
proj_dir = '/home/birniece/Documents/Projects/MicroseismicModelling/SOFI_LargeRockBlock_Sandstone/expSetUp'

In [4]:
# Project Files - 40 cm Cube -> No boundaries
rec_file = './inputs/RecArray_expSetUp_wBounds_1mm.dat'
model_dir = './inputs/expSetUp_wBounds_xyz_1mm'
src_file = './inputs/SourceArray_singleshots_expSetUp_wBounds_1mm.dat'

# MODELLING PARAMS (Indep. of src locs)

### MODEL PARAMETERS - Get these from the script that made the models

In [5]:
fc_kHz = 25  # kHz
fs_kHz = 100

fc = fc_kHz * 1000
fs = fs_kHz * 1000
rec_dt = 1/fs
print(f"Event Central Frequency: {fc},\nSampling Frequency: {fs},\nReceiver dt: {rec_dt}")

# time params (in seconds!)
dt = 5e-8
tdur = 0.001
tsnap_params = [dt, dt*2000, dt*50]


Event Central Frequency: 25000,
Sampling Frequency: 100000,
Receiver dt: 1e-05


In [6]:
dx = dy = dz = 0.001 
n_xzy = (504, 452, 504)
d_xzy = [dx, dz, dy]


In [7]:
# BOUNDARIES & FREE SURFACE
nbounds = 30
free_surface = True

In [8]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur/8
snap_step = 50*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 2

# SEISMOGRAM PARAMETERS
smgrm_ndt = 10

In [9]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              ' ',
                              moddir=model_dir,
                              np_xzy=[2,2,2]
                             )
bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=True)


In [10]:
groupname = "expSetUp"

# Make time string (long as multishot in one rec)
t_str = get_time_str(dt, tdur)

# Make SOFI Source String
src_str = get_source_str(sfile=src_file, multisource=0)

# Make Monitor String
monitor_strs = get_monitor_str(tsnap_params, 
                           smgrm_ndt, 
                           groupname, 
                           sbsmp_xyz=snap_sbsmp_xyz,
                           snap=True
                          )

# Write SOFI FILE
sofi_json_file = os.path.join(proj_dir, groupname+'.json')
write_SOFIjsonParams(default_strs, 
                     monitor_strs,  
                     t_str, 
                     mod_strs, 
                     src_str, 
                     bndry_str,
                     sofi_json_file)



'#-----------------------------------------------------------------\n#      JSON PARAMETER FILE FOR SOFI3D\n#-----------------------------------------------------------------\n# description: example of json input file\n# description/name of the model: homogeneous full space (hh.c)\n#\n\n \n{\n \n"Checkpoints" : "comment",\n    "CHECKPTREAD" : "0",\n    "CHECKPTWRITE" : "0",\n    "CHECKPT_FILE" : "tmp/checkpoint_sofi3D",\n \n"FD order" : "comment",\n    "FDORDER" : "4",\n    "FDORDER_TIME" : "2",\n    "FDCOEFF" : "2",\n    "fdcoeff values: Taylor=1, Holberg=2" : "comment",\n \n"Q-approximation" : "comment",\n    "L" : "0",\n    "FREF" : "5.0",\n    "FL1" : "5.0",\n    "TAU" : "0.00001",\n \n"Receiver" : "comment",\n    "SEISMO" : "4",\n    "READREC" : "1",\n    "REC_FILE" : "./inputs/RecArray_expSetUp_wBounds_1mm.dat",\n    "REFRECX, REFRECY, REFRECZ" : "0.0 , 0.0 , 0.0",\n    "NGEOPH" : "1",\n    "REC_ARRAY" : "0", # No array as it is read from the rec file\n \n"Snapshots" : "comment",

# MAKE BASH SCRIPT TO SUBMIT ALL SLURM JOBS

In [11]:
bashsubmitfile = os.path.join(proj_dir, 'submit_ALL.sh')

bashsubmitstring = '''#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

mpirun -np 8 %s %s > sofi3D.jout

# Do the snap merge thing
snapmergepath=~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/snapmerge
${snapmergepath} %s

# Clean up the snap files (for memory purposes)
rm -rf ./outputs/snap/**.0*
rm -rf ./outputs/snap/**.1*
'''%(sofimaster,sofi_json_file,sofi_json_file)

text_file = open(bashsubmitfile, "wt")
text_file.writelines(bashsubmitstring)
text_file.close()